<a href="https://colab.research.google.com/github/Ericlinyuting/Web_Crawler_591/blob/main/newhouse591_without_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

In [18]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import datetime
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
import io
import json
import random
import math
warnings.filterwarnings("ignore")

In [4]:
headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }

request_url='http://newhouse.591.com.tw/home/housing/info?hid=119282'
res=requests.get(request_url, headers = headers)
bs=BeautifulSoup(res.text,'html.parser')


In [28]:
# input 建案網址 return 建案建案詳情欄位
# 產出為建案名與16個建案資訊
def getData(url):
    request_url='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()
    headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
    res=requests.get(request_url, headers = headers)
    #bs=BeautifulSoup(res.text,'html.parser')

    request_url_detail='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()+"/detail"
    res_detail=requests.get(request_url_detail, headers = headers)

    if res.status_code == 200 & res_detail.status_code==200:
        bs=BeautifulSoup(res.text,'html.parser')
        bs_detail=BeautifulSoup(res_detail.text,'html.parser')
        #先宣告變數為NULL 若無撈到資料則寫入NULL
        title="NULL"

        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
        # 利用 try except 若無撈到資料則寫入NULL
        title=bs.find('h1').text
        try:
          tag = ', '.join([span.text.strip() for span in bs.find("p", {'class':"build-tag"}).find_all("span")]) if bs.find("p", {'class':"build-tag"}) else "未找到相應標籤" #建案標籤
        except:
          tag = "NULL"
        try:
          unit_price=bs.find("span", {'class':"price"}).text #單價
        except:
          unit_price= "NULL"
        try:
          unit = bs.find("span", {'class':"unit"}).text #單位
        except:
          unit = "NULL"
        try:
          material = bs.find("h4",text="建材說明").findNext("p").text #建材說明
        except:
          material= "NULL"
        try:
          htype=bs_detail.find("span", text="建案類別").findNext("p").string.strip() #建案類別
        except:
          htype= "NULL"
        try:
          htype2=bs_detail.find("span", text="建案型態").findNext("p").string.strip().replace(' ', '').replace('\n', '、') #建案型態
        except:
          htype2= "NULL"
        try:
          htype3=bs_detail.find("span", text="公開銷售").findNext("p").string.strip() #公開銷售
        except:
          htype3= "NULL"
        try:
          htype4=bs_detail.find("p", {'class':"address"}).findNext("span").text #基地地址
        except:
          htype4= "NULL"
        try:
          htype5=bs_detail.find("span", text="交屋屋況").findNext("p").string.strip() #交屋屋況
        except:
          htype5= "NULL"
        try:
          htype6=bs_detail.find("span",text="格局規劃").findNext("p").text.strip().replace(' ', '') #格局規劃
        except:
          htype6= "NULL"
        try:
          htype7=bs_detail.find("span", text="投資建設").findNext("p").string.strip() #投資建設
        except:
          htype7= "NULL"
        try:
          htype8=bs_detail.find("span", text="營造公司").findNext("p").string.strip() #營造公司
        except:
          htype8= "NULL"
        try:
          htype9=bs_detail.find("span", text="棟戶規劃").findNext("p").string.strip() #棟戶規劃
        except:
          htype9= "NULL"
        try:
          htype10=bs_detail.find("span", text="樓層規劃").findNext("p").string.strip() #樓層規劃
        except:
          htype10= "NULL"
        try:
          htype11=bs_detail.find("span", text="用途規劃").findNext("p").string.strip() #用途規劃
        except:
          htype11= "NULL"
        return title,tag,unit_price,unit,material,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11
    else:
        print('link expired:', url)
        return 404, 404, 404, 404, 404, 404, 404

In [20]:
#def get_dynamic_data(url):
#    request_url='https://newhouse.591.com.tw/home/housing/dynamic?hid='+str(url).strip()
#    headers={
#    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
#    }
#    res=requests.get(request_url, headers = headers)
#    #bs=BeautifulSoup(res.text,'html.parser')
#
#    if res.status_code == 200:
#        bs=BeautifulSoup(res.text,'html.parser')
#        #先宣告變數為NULL 若無撈到資料則寫入NULL
#        news_date = 'NULL'
#        news = 'NULL'
#        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
#        news_date = bs.find("div", {'class':"dynamic-date"}).findNext("span").contents[0].string.strip() + "-" +\
#                    bs.find("div", {'class':"dynamic-date"}).findNext("strong").contents[0].string.strip()
#
#        news = bs.find("div", {'class':"dynamic-infolist"}).string
#
#        return news_date, news
#    else:
#        print('link expired:', url)
#        return 404, 404, 404, 404, 404, 404, 404

In [19]:
def main(outputfile, rid, sid, totalpages):

   with io.open(outputfile, "w", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([ "建案名稱", "建案標籤","單價","單位","建材說明" ,"建案類別", "建物形態", "公開銷售", "基地地址", "交屋屋況"\
                        ,"格局規劃", "投資建設", "營造公司", "棟戶規劃", "樓層規劃", "用途規劃", "網址"]) \
                        #,"動態資訊日期日期", "動態資訊"])
        totalpages = totalpages
        print('Total pages: ', totalpages)

        for i in range(1, totalpages+1):
            request_url = "https://newhouse.591.com.tw/home/housing/search?rid="+str(rid)+"&sid="+str(sid)+"&page="+str(i)
            response = requests.get(request_url, headers = headers)
            response = response.json()
            items = response["data"]["items"]

            house_url_list=[] #存放網址list
            for key in items:
                id = key["hid"] # 每個物件的 id
                house_url_list.append(id)
            #if totalpages%14 == 0:
            #  time.sleep(180)
            #else:
            time.sleep(random.randint(5,100)) #反爬蟲

            # ------------- write into csv ------------- #
            for url in house_url_list:
                title,tag,unit_price,unit,material,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11 = getData(url)
                #news_date, news = get_dynamic_data(url)
                writer.writerow([title,tag,unit_price,unit,material,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11,\
                                 str('https://newhouse.591.com.tw/home/housing/info?hid='+str(url))])
                                 #, news_date, news])
                #print("url=",str('https://newhouse.591.com.tw/home/housing/info?hid='+str(url)))
            #print("rid=",rid,",sid=",sid,",page=",i,)
            # ------------------------------------------ #
            print(i/totalpages*100, '%') # print out 完成 %數

In [26]:
if __name__ == '__main__':
    # -------- configurable parameter -------- #
    # 以台北市不限區舉例(預設網址可能沒寫rid&sid, 點選縣市或區往只會顯示如下)
    # link:https://newhouse.591.com.tw/housing-list.html?rid=1&sid=0

    request_url = "https://newhouse.591.com.tw/home/housing/search?rid=0&sid=0"
    headers={
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
        }
    response = requests.get(request_url, headers = headers)
    response = response.json()
    total = response["data"]["total"]
    per_page = response["data"]["per_page"]

    output_file_name = '/content/drive/MyDrive/output.csv' #設定存放位置與檔名
    rid = 0           # 設定縣市 (台北市 rid = 1)
    sid = 0           # 設定地區 (不限區 sid = 0)
    totalpages = math.ceil(total/per_page)    # 設定抓取頁數
    # ---------------------------------------- #

    main(output_file_name, rid, sid, totalpages)                                          #匯出csv檔
    read_file = pd.read_csv (output_file_name)
    read_file.to_excel ('/content/drive/MyDrive/output.xlsx', index = None, header=True) #匯出excel檔
    print('\nfinish!')

Total pages:  782
rid= 0 ,sid= 0 ,page= 1
0.1278772378516624 %
rid= 0 ,sid= 0 ,page= 2
0.2557544757033248 %
rid= 0 ,sid= 0 ,page= 3
0.3836317135549872 %
rid= 0 ,sid= 0 ,page= 4
0.5115089514066496 %
rid= 0 ,sid= 0 ,page= 5
0.639386189258312 %
rid= 0 ,sid= 0 ,page= 6
0.7672634271099744 %
rid= 0 ,sid= 0 ,page= 7
0.8951406649616368 %
rid= 0 ,sid= 0 ,page= 8
1.0230179028132993 %
rid= 0 ,sid= 0 ,page= 9
1.1508951406649617 %
rid= 0 ,sid= 0 ,page= 10
1.278772378516624 %
rid= 0 ,sid= 0 ,page= 11
1.4066496163682864 %
rid= 0 ,sid= 0 ,page= 12
1.5345268542199488 %
rid= 0 ,sid= 0 ,page= 13
1.6624040920716114 %
rid= 0 ,sid= 0 ,page= 14
1.7902813299232736 %
rid= 0 ,sid= 0 ,page= 15
1.9181585677749362 %
rid= 0 ,sid= 0 ,page= 16
2.0460358056265986 %
rid= 0 ,sid= 0 ,page= 17
2.1739130434782608 %
rid= 0 ,sid= 0 ,page= 18
2.3017902813299234 %
rid= 0 ,sid= 0 ,page= 19
2.4296675191815855 %
rid= 0 ,sid= 0 ,page= 20
2.557544757033248 %
rid= 0 ,sid= 0 ,page= 21
2.6854219948849107 %
rid= 0 ,sid= 0 ,page= 22
2.8

ValueError: ignored